### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "stopwords.txt"
with open(filename) as f:
    lines = f.readlines()

stopwords = []
for line in lines:
    line = line.strip()
    stopwords.append(line)

In [5]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器


filename = "case_summary1.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    ne_list = []
    for term in main_list:
        terms_list = pynlpir.segment(term)
        for t in terms_list:
            ne_list.append(t[0])
            
    #print(ne_list)
    
    word_list = []
    
    if idx in [64,223,251,389,480,673,744,751,769,1422,1541,1853]:
        continue
    
    txt_list = pynlpir.segment(case_content)
    #print(txt_list)
    for txt in txt_list:
        if txt in stopwords:
            continue
        
        if txt[0] in ne_list:
            e = (txt[0],txt[1],"S")
        else :
            e = (txt[0],txt[1],"O")
        word_list.append(e)
        
    docs.append(word_list)

print(len(docs))   

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: '

2095


In [8]:
data = docs

In [6]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    if postag is None:
            postag = 'none'
    
    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [9]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train[0][4]))
print(X_train[0][5])

In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 60769
Seconds required: 1.635

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 268931.519425
Feature norm: 1.000000
Error norm: 162279.587039
Active features: 59757
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 0.411

***** Iteration #2 *****
Loss: 254117.269518
Feature norm: 0.900533
Error norm: 126106.061540
Active features: 58561
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.217

***** Iteration #3 *****
Loss: 246329.500378
Feature norm: 0.577659
Error norm: 194196.007508
Active features: 40666
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #40 *****
Loss: 105797.192894
Feature norm: 101.294727
Error norm: 5439.178656
Active features: 33598
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.237

***** Iteration #41 *****
Loss: 104787.540093
Feature norm: 113.838805
Error norm: 8494.130186
Active features: 32692
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.257

***** Iteration #42 *****
Loss: 104030.646459
Feature norm: 122.995268
Error norm: 4104.905563
Active features: 32437
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #43 *****
Loss: 103214.137403
Feature norm: 136.002299
Error norm: 7083.616176
Active features: 32450
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.217

***** Iteration #44 *****
Loss: 102536.056830
Feature norm: 145.180912
Error norm: 4111.170207
Active features: 32146
Line search trials: 1
Line search step

***** Iteration #80 *****
Loss: 98025.724287
Feature norm: 207.967673
Error norm: 1623.665150
Active features: 27833
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.214

***** Iteration #81 *****
Loss: 98010.219304
Feature norm: 208.098774
Error norm: 3282.786553
Active features: 27777
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.209

***** Iteration #82 *****
Loss: 97992.515334
Feature norm: 208.126330
Error norm: 1160.565727
Active features: 27773
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.208

***** Iteration #83 *****
Loss: 97979.840598
Feature norm: 208.148328
Error norm: 2183.807309
Active features: 27710
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.211

***** Iteration #84 *****
Loss: 97961.988317
Feature norm: 208.175097
Error norm: 1556.189536
Active features: 27650
Line search trials: 1
Line search step: 1.0

***** Iteration #119 *****
Loss: 97664.132016
Feature norm: 209.389246
Error norm: 2479.034574
Active features: 26733
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.211

***** Iteration #120 *****
Loss: 97656.567261
Feature norm: 209.378389
Error norm: 1526.254836
Active features: 26729
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.208

***** Iteration #121 *****
Loss: 97650.948378
Feature norm: 209.343300
Error norm: 1224.440956
Active features: 26708
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.207

***** Iteration #122 *****
Loss: 97645.372156
Feature norm: 209.316153
Error norm: 1444.044839
Active features: 26692
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.207

***** Iteration #123 *****
Loss: 97641.085709
Feature norm: 209.243442
Error norm: 2360.515588
Active features: 26664
Line search trials: 1
Line search step

***** Iteration #161 *****
Loss: 97475.016951
Feature norm: 207.289608
Error norm: 2107.166005
Active features: 26133
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.211

***** Iteration #162 *****
Loss: 97469.839949
Feature norm: 207.258446
Error norm: 1740.185405
Active features: 26124
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.210

***** Iteration #163 *****
Loss: 97464.996091
Feature norm: 207.201101
Error norm: 1775.784459
Active features: 26115
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.209

***** Iteration #164 *****
Loss: 97460.053337
Feature norm: 207.176180
Error norm: 1684.816839
Active features: 26108
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.208

***** Iteration #165 *****
Loss: 97456.255890
Feature norm: 207.120567
Error norm: 2025.343311
Active features: 26087
Line search trials: 1
Line search step

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [13]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "B-S"]))

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "B-S"]))

             precision    recall  f1-score   support

          O       0.97      0.98      0.97    891522
        B-S       0.72      0.55      0.62     69848

avg / total       0.95      0.95      0.95    961370

             precision    recall  f1-score   support

          O       0.96      0.98      0.97    217862
        B-S       0.66      0.46      0.54     18178

avg / total       0.93      0.94      0.94    236040

